<a href="https://colab.research.google.com/github/r2x4/prueba_cac_abogados/blob/main/cac_abogados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prueba CAC Abogados**

In [173]:
# llamar datos

import pandas as pd

datos = pd.read_csv('/content/datos1.csv', encoding='latin1')


In [174]:
datos.sample(7)

,FECHA_EVOLUCION,IDENTIFICACION,NOMBRE_TITULAR,SALDO_TOTAL_CLIENTE,RXM_ACTUAL,TIPO_CLIENTE,LINEA_CLIENTE,LINEA_PRODUCTO,MACROPORTAFOLIO,EXM_INICIAL,Fecha_asignacion
494134,31/10/2024,7.978769e+07,HERRERA GONZALEZ GUSTALVO ADOL,5625983.0,30.0,Clasico,CS,CS,MULTICS,E60,2024/10
415653,31/07/2024,8.402762e+07,PALACIO PIMIENTA CARLOS MARTIN,50873971.0,9999.0,Clasico,CH,CH,TITULARIZADO,E>120,2024/07
508392,30/09/2024,3.431794e+07,DAVILA ESCOBAR DIANA EUGENIA,185484054.0,9999.0,n.d,CS,CS,MULTICS,CASTIGO,2024/09
121317,29/02/2024,1.130631e+09,CASTRO MUNOZ DEBBIE ALEJANDRA,200518526.0,0.0,n.d,CH,CH,HIPOTECARIO,MIXTO,2024/02
288594,30/06/2024,1.107509e+09,GARRIDO URBANO MARILYN DANIELA,37232336.0,9999.0,Clasico,CS,CS,MULTICS,E>120,2024/06
369118,31/07/2024,3.090381e+06,ANGEL ORJUELA S,3526296.0,0.0,n.d,CS,NaN,MULTICS,E120,2024/07
261486,31/05/2024,7.992103e+07,MONTEALEGRE CIFUENTES FRANKLIN,124424427.0,150.0,Clasico,CS,CS,MULTICS,E120,2024/05


In [175]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664293 entries, 0 to 664292
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FECHA_EVOLUCION      664292 non-null  object 
 1   IDENTIFICACION       664292 non-null  float64
 2   NOMBRE_TITULAR       664112 non-null  object 
 3   SALDO_TOTAL_CLIENTE  664112 non-null  float64
 4   RXM_ACTUAL           650871 non-null  float64
 5   TIPO_CLIENTE         664292 non-null  object 
 6   LINEA_CLIENTE        664292 non-null  object 
 7   LINEA_PRODUCTO       520154 non-null  object 
 8   MACROPORTAFOLIO      664292 non-null  object 
 9   EXM_INICIAL          664292 non-null  object 
 10  Fecha_asignacion     664293 non-null  object 
dtypes: float64(3), object(8)
memory usage: 55.7+ MB


In [176]:
# datos nulos
datos.isnull().sum()

,0
FECHA_EVOLUCION,1
IDENTIFICACION,1
NOMBRE_TITULAR,181
SALDO_TOTAL_CLIENTE,181
RXM_ACTUAL,13422
TIPO_CLIENTE,1
LINEA_CLIENTE,1
LINEA_PRODUCTO,144139
MACROPORTAFOLIO,1
EXM_INICIAL,1


In [177]:
datos.describe()

,IDENTIFICACION,SALDO_TOTAL_CLIENTE,RXM_ACTUAL
count,6.642920e+05,6.641120e+05,650871.000000
mean,4.715052e+08,1.180369e+08,4467.062114
std,5.036787e+08,1.588102e+08,4929.063091
min,2.522000e+04,0.000000e+00,0.000000
25%,4.166254e+07,1.783153e+07,60.000000
50%,8.065687e+07,6.450916e+07,180.000000
75%,1.045744e+09,1.568415e+08,9999.000000
max,2.000011e+09,1.254467e+10,9999.000000


In [178]:
datos.dtypes

,0
FECHA_EVOLUCION,object
IDENTIFICACION,float64
NOMBRE_TITULAR,object
SALDO_TOTAL_CLIENTE,float64
RXM_ACTUAL,float64
TIPO_CLIENTE,object
LINEA_CLIENTE,object
LINEA_PRODUCTO,object
MACROPORTAFOLIO,object
EXM_INICIAL,object


In [179]:
datos.shape

(664293, 11)